mlflow server --host 0.0.0.0 --port 5000

In [1]:
%env CUDA_VISIBLE_DEVICES=MIG-3b133eb8-ed94-51f8-937f-cbc3e3f3ff2a
#MIG-4f53211e-ceab-5aa7-b0ba-6ef87e62c8ed
%env CUDA_VISIBLE_DEVICES

env: CUDA_VISIBLE_DEVICES=MIG-3b133eb8-ed94-51f8-937f-cbc3e3f3ff2a


'MIG-3b133eb8-ed94-51f8-937f-cbc3e3f3ff2a'

In [2]:
import argparse
from pathlib import Path
from datetime import datetime

import mlflow
from monai.losses import DiceCELoss
from torch.utils.tensorboard import SummaryWriter
import torch

import cfg
from dataset_mevis_v2 import MRI_dataset_batched
from models.sam import build_sam_mevis
from retrain_v2 import lr_scheduler, train_mevis, validate_mevis, write_hists

# TODO:
# run signature with bone-sam address and time

/home/ubuntu/Thesis_code/models/sam/modeling/tiny_vit_sam.py:677: UserWarning: Overwriting tiny_vit_5m_224 in registry with models.sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/ubuntu/Thesis_code/models/sam/modeling/tiny_vit_sam.py:677: UserWarning: Overwriting tiny_vit_11m_224 in registry with models.sam.modeling.tiny_vit_sam.tiny_vit_11m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/ubuntu/Thesis_code/models/sam/modeling/tiny_vit_sam.py:677: UserWarning: Overwriting tiny_vit_21m_224 in registry with models.sam.modeling.tiny_vit_sam.tiny_vit_21m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
/home/ubuntu/Thesis_code/mo

In [3]:
def mevis_args_parser():
    parser = argparse.ArgumentParser(description="Mevis SAM fine-tuning parameters.")
    parser.add_argument(
        "--checkpoint_path",
        type=str,
        default="",
        help="Checkpoint to continue the training",
    )
    parser.add_argument(
        "--prompt_probability",
        type=float,
        default=0.3,
        help="probability of generating prompts for each batch",
    )
    parser.add_argument(
        "--lr_schedule",
        type=bool,
        default=True,
        help="Use earning rate scheduler during training.",
    )
    parser.add_argument(
        "--lr_train_start",
        type=float,
        default=5e-4,
        help="Learning rate starting value during training.",
    )
    parser.add_argument(
        "--lr_train_end",
        type=float,
        default=5e-5,
        help="Learning rate ending value during training.",
    )
    parser.add_argument(
        "--lr_warmup", type=float, default=1e-5, help="Learning rate on warmup."
    )
    parser.add_argument(
        "--epochs", type=int, default=100, help="Number of training epochs"
    )
    parser.add_argument(
        "--epochs_warmup", type=int, default=20, help="Number of warmup epochs"
    )
    parser.add_argument(
        "--batch_size", type=int, default=120, help="Number of slices in a batch"
    )
    mevis_args = parser.parse_args("")

    return mevis_args


mevis_args = mevis_args_parser()
args = cfg.parse_args()

In [4]:
tracking_server = "http://0.0.0.0:5000"
TRAIN_DATA_FILE = "data_files/Train_data_files.json"
VALID_DATA_FILE = "data_files/Validation_data_files.json"
DEVICE = torch.device("cuda:" + str(args.gpu_device))
CHECKPOINT_DIRECTORY = Path("/data/sab_data/checkpoints")
LOGDIR = Path("/data/sab_data/model_logs/mlflow")
# train_folder = "decoder_only/30p_prompt_v2"
BONE_CHECKPOINT = CHECKPOINT_DIRECTORY / "bone_sam.pth"
VALID_EVERY = 2
SAVE_EVERY = 3
OPEN_LAYERS = (
    "mask_decoder.transformer.layers.0.MLP_Adapter,"
    "mask_decoder.transformer.layers.0.Adapter,"
    "mask_decoder.transformer.layers.1.MLP_Adapter,"
    "mask_decoder.transformer.layers.1.Adapter,"
)
merged_args = argparse.Namespace(**vars(args), **vars(mevis_args))

In [5]:
experiment_name = "sample_name"
experiment_description = (
    "test test This is fine-tuning of SAB model for vertebrae. "
    "The improvements from first version are applied."
)

# Provide searchable tags that define characteristics of the Runs that
# will be in this Experiment
experiment_tags = {
    "project_name": "new_proj",
    "version": "9.0",
    "project_time": "October-2024",
    "opened_layers": OPEN_LAYERS,
    "mlflow.note.content": experiment_description,
}

# Create the Experiment, providing a unique name
mlflow.set_tracking_uri(uri=tracking_server)
experiment = mlflow.get_experiment_by_name(experiment_name)
if experiment is None:
    mlflow.create_experiment(
        name=experiment_name,
        tags=experiment_tags,
    )
    experiment = mlflow.set_experiment(experiment_name)
log_base = LOGDIR / experiment.experiment_id

In [6]:
dataset_validation = MRI_dataset_batched(
        merged_args,
        data_file=VALID_DATA_FILE,
        batch_size=mevis_args.batch_size,
        phase="test",
        operation_mode="queue",
        mask_out_size=merged_args.out_size,
        attention_size=64,
        crop=False,
        crop_size=1024,
        cls=1,
        if_prompt=True,
        prompt_type="points",
        if_attention_map=True,
        device=DEVICE,
    )

In [7]:
for i in range(len(dataset_validation)):
    batch = dataset_validation[i]
    if batch.get("points", None) is not None:
        points = batch["points"]
        print(batch["points"].shape)
        break

torch.Size([120, 1, 2])


In [8]:
batch.keys()

dict_keys(['slices', 'original_size', 'image_name', 'images', 'masks', 'atten_maps', 'cat_indexes', 'points', 'point_labels'])

In [9]:
from models.sam import sam_model_registry
from models.sam.modeling.prompt_encoder import Attention_Fusion

def forward(
        self,
        batched_input: dict,
        multimask_output: bool = True,
        if_attention=False,
    ) -> dict[str, torch.Tensor]:
        """
        Predicts masks end-to-end from provided images and prompts.
        If prompts are not known in advance, using SamPredictor is
        recommended over calling the model directly.

        Arguments:
            batched_input (dict): A dictionary with the following keys. A prompt key can be
            excluded if it is not present.
                'images': The image as a torch tensor in 3xHxW format,
                already transformed for input to the model.
                'points': (torch.Tensor) Batched point prompts for
                this image, with shape BxNx2. Already transformed to the
                input frame of the model.
                'point_labels': (torch.Tensor) Batched labels for point prompts,
                with shape BxN.
                'atten_maps": (optional)(torch.Tensor) Batched attention maps, with
                shape Bx64x64.
                'boxes': (torch.Tensor) Batched box inputs, with shape Bx4.
                Already transformed to the input frame of the model.
                'mask_inputs': (torch.Tensor) Batched mask inputs to the model,
                in the form Bx1xHxW.
            multimask_output (bool): Whether the model should predict multiple
            disambiguating masks, or return a single mask.

        Returns:
            (list(dict)): A list over input images, where each element is
            as dictionary with the following keys.
                'masks': (torch.Tensor) Batched binary mask predictions,
                with shape BxCxHxW, where B is the number of input prompts,
                C is determined by multimask_output, and (H, W) is the
                original size of the image.
        """
        with torch.no_grad():
            image_embeddings = self.image_encoder(batched_input["images"])
            if if_attention:
                image_embeddings = self.attention_fusion(
                    image_embeddings, batched_input["atten_maps"].unsqueeze(1)
                )

            if "points" in list(batched_input.keys()):
                points = (batched_input["points"], batched_input["point_labels"])
            else:
                points = None

            sparse_embeddings, dense_embeddings = self.prompt_encoder(
                points=points,
                boxes=batched_input.get("boxes", None),
                masks=batched_input.get("mask_inputs", None),
            )
            pe = torch.stack(
                [self.prompt_encoder.get_dense_pe() for _ in range(image_embeddings.shape[0])],
                dim=0,
            ).squeeze(1)
            ###
            print("sam_mevis", pe.shape)
            ###
            
        low_res_masks, iou_predictions = self.mask_decoder(
            image_embeddings=image_embeddings,
            image_pe=pe,
            sparse_prompt_embeddings=sparse_embeddings,
            dense_prompt_embeddings=dense_embeddings,
            multimask_output=multimask_output,
        )

        return low_res_masks

sam_mevis = sam_model_registry["vit_t"](
    args, checkpoint=CHECKPOINT_DIRECTORY / "mobile_sam.pt", num_classes=2
)
sam_mevis.attention_fusion = Attention_Fusion()
sam_mevis.load_state_dict(
    torch.load(
        CHECKPOINT_DIRECTORY / BONE_CHECKPOINT,
        map_location=torch.device(DEVICE),
    ),
    strict=True,
)
# sam_mevis.forward = forward
sam_mevis = sam_mevis.to(DEVICE)

/home/ubuntu/Thesis_code/models/sam/build_sam.py:113: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(f)
/tmp/ipykernel_125483/51230101.py:82: FutureWa

In [9]:
sam_mevis = build_sam_mevis(
        merged_args,
        mevis_checkpoint=CHECKPOINT_DIRECTORY / BONE_CHECKPOINT,
        num_classes=2,
        device=DEVICE,
    )
open_layers = [x for x in OPEN_LAYERS.split(",") if x != ""]
for param in sam_mevis.parameters():
    param.requires_grad = False
for name, mod in sam_mevis.named_parameters():
    for l in open_layers:
        if l in name:
            mod.requires_grad = True

In [12]:
imgs = batch["images"]
masks = batch["masks"]
attens = batch["atten_maps"]
pt = (batch["points"], batch["point_labels"])
with torch.no_grad():
    imge = sam_mevis.image_encoder(imgs)
    se, de = sam_mevis.prompt_encoder(
        points=pt,
        boxes=None,
        masks=None,
    )
    pe = torch.stack(
        [sam_mevis.prompt_encoder.get_dense_pe() for _ in range(imge.shape[0])],
        dim=0,
    ).squeeze(1)
    print(
        pe.shape,
        torch.stack(
            [sam_mevis.prompt_encoder.get_dense_pe() for _ in range(imge.shape[0])],
            dim=0,
        ).shape,
    )
    imge = sam_mevis.attention_fusion(imge, attens.unsqueeze(1))
pred, _ = sam_mevis.mask_decoder(
    image_embeddings=imge,
    image_pe=pe,
    sparse_prompt_embeddings=se,
    dense_prompt_embeddings=de,
    multimask_output=True,
)

torch.Size([120, 256, 64, 64]) torch.Size([120, 1, 256, 64, 64])
mask_decoder torch.Size([14400, 256, 64, 64]) torch.Size([120, 256, 64, 64]) torch.Size([120, 256, 64, 64]) torch.Size([120, 256, 64, 64]) torch.Size([120, 6, 256])
transformer torch.Size([120, 6, 256]) torch.Size([120, 4096, 256]) torch.Size([14400, 4096, 256]) torch.Size([120, 6, 256])


RuntimeError: The size of tensor a (120) must match the size of tensor b (14400) at non-singleton dimension 0

In [11]:
sam_mevis.forward(batched_input=batch, multimask_output=True, if_attention=True)

sam_mevis torch.Size([1, 256, 64, 64])
mask_decoder torch.Size([120, 256, 64, 64]) torch.Size([120, 256, 64, 64]) torch.Size([1, 256, 64, 64]) torch.Size([120, 256, 64, 64]) torch.Size([120, 6, 256])
transformer torch.Size([120, 6, 256]) torch.Size([120, 4096, 256]) torch.Size([120, 4096, 256]) torch.Size([120, 6, 256])


tensor([[[[  4.6366,   4.6843,   4.7206,  ...,   4.9741,   4.7001,   5.1016],
          [  4.5377,   5.8957,   4.8945,  ...,   5.6354,   5.0819,   5.1050],
          [  4.8227,   5.3301,   5.1379,  ...,   5.4952,   4.8372,   4.7834],
          ...,
          [  4.2775,   5.3811,   4.7832,  ...,   4.9188,   4.5895,   4.7840],
          [  5.3820,   5.1543,   5.1019,  ...,   5.0971,   4.7584,   5.2836],
          [  5.1297,   5.7867,   4.9409,  ...,   5.3520,   4.9922,   5.9886]],

         [[ -9.5473,  -9.4737,  -9.1879,  ...,  -7.4058,  -7.3565,  -7.9861],
          [ -9.5368,  -9.8512,  -9.3096,  ...,  -7.0785,  -7.7052,  -7.6426],
          [-10.1158, -10.7048, -10.8573,  ...,  -7.9592,  -8.0751,  -7.7347],
          ...,
          [-10.1512, -10.3841, -11.2979,  ...,  -7.7637,  -8.3725,  -8.4147],
          [-12.4639, -12.9783, -12.8968,  ...,  -9.2074, -10.5699, -10.4548],
          [-13.0393, -13.9144, -12.6940,  ..., -10.2157, -11.1949, -12.8792]]],


        [[[  4.5127,   4.523

In [6]:
def train_procedure(
    run,
    epoch,
    log_base,
    mevis_args,
    merged_args,
):
    
    run_id = run.info.run_id
    # Tensorboard logger
    log_save = log_base / run_id
    log_save.mkdir(parents=True, exist_ok=True)
    writer = SummaryWriter(log_dir=log_save)

    # Log the hyperparameters
    mlflow.log_params(vars(mevis_args))

    # data
    dataset_validation = MRI_dataset_batched(
        merged_args,
        data_file=VALID_DATA_FILE,
        batch_size=mevis_args.batch_size,
        phase="test",
        operation_mode="queue",
        mask_out_size=merged_args.out_size,
        attention_size=64,
        crop=False,
        crop_size=1024,
        cls=1,
        if_prompt=True,
        prompt_type="points",
        if_attention_map=True,
        device=DEVICE,
    )
    dataset_train = MRI_dataset_batched(
        merged_args,
        data_file=TRAIN_DATA_FILE,
        batch_size=mevis_args.batch_size,
        phase="train",
        operation_mode="queue",
        mask_out_size=merged_args.out_size,
        attention_size=64,
        crop=False,
        crop_size=1024,
        cls=1,
        if_prompt=True,
        prompt_type="points",
        if_attention_map=True,
        device=DEVICE,
    )

    # model
    sam_mevis = build_sam_mevis(
        merged_args,
        mevis_checkpoint=CHECKPOINT_DIRECTORY / BONE_CHECKPOINT,
        num_classes=2,
        device=DEVICE,
    )
    open_layers = [x for x in OPEN_LAYERS.split(",") if x != ""]
    for param in sam_mevis.parameters():
        param.requires_grad = False
    for name, mod in sam_mevis.named_parameters():
        for l in open_layers:
            if l in name:
                mod.requires_grad = True

    total_trainable_params = sum(
        p.numel() for p in sam_mevis.parameters() if p.requires_grad
    )
    mlflow.log_param("total_trainable_params", total_trainable_params)
    print("Number of trainable parameters in the model:", total_trainable_params)

    lossfunc = DiceCELoss(sigmoid=True, squared_pred=True, reduction="mean")
    optimizer = torch.optim.AdamW(sam_mevis.parameters(), lr=mevis_args.lr_train_start)

    while epoch < (merged_args.lr_warmup + merged_args.epochs):
        optimizer = lr_scheduler(
            optimizer,
            epoch,
            n_warmup_epochs=merged_args.epochs_warmup,
            n_epochs=merged_args.epochs,
            base_lr=merged_args.lr_train_start,
            warmup_lr_start=merged_args.lr_warmup,
            end_lr=merged_args.lr_train_end,
        )
        dataset_train.on_epoch_end()
        loss = train_mevis(
            net=sam_mevis,
            optimizer=optimizer,
            loss_func=lossfunc,
            dataset=dataset_train,
            epoch=epoch,
            writer=writer,
        )
        mlflow.log_metric(key="DiceCE", value=loss, step=epoch)
        writer.add_scalars("DiceCEloss", {"training": loss}, epoch)
        write_hists(sam_mevis, writer=writer, epoch=epoch)
        if epoch % VALID_EVERY == 0:
            val_loss, val_result = validate_mevis(
                net=sam_mevis,
                loss_func=lossfunc,
                dataset=dataset_validation,
                epoch=epoch,
                writer=writer,
            )
            writer.add_scalars("DiceCEloss", {"validation": val_loss}, epoch)
            for key, value in val_result.items():
                for key_in, value_in in value.items():
                    writer.add_scalar(
                        f"Validation Metrics/{key_in}/thr{key}", value_in, epoch
                    )
        if epoch % SAVE_EVERY == 0:
            mlflow.pytorch.log_model(
                sam_mevis, artifact_path=f"mevis_sam-epoch{epoch}-{run_id}"
            )

        epoch += 1

    mlflow.pytorch.log_model(
        sam_mevis, artifact_path=f"mevis_sam-epoch{epoch}-{run_id}"
    )
    writer.close()

In [7]:
if mevis_args.checkpoint_path != "":
    new_path = Path(mevis_args.checkpoint_path)
    if new_path.is_file() and new_path.suffix == ".pth":
        BONE_CHECKPOINT = new_path

if "bone_sam" in BONE_CHECKPOINT.name:
    epoch = 0
    run_id = None
else:
    epoch = int(BONE_CHECKPOINT.name.split("epoch")[1].split("-")[0])
    run_id = BONE_CHECKPOINT.name.split("-")[-1]
# run_id = "putting_into_test"

mlflow.set_experiment(experiment_name=experiment_name)
if run_id is not None:
    with mlflow.start_run(run_id=run_id) as parent:
        train_procedure(
            parent,
            epoch,
            log_base=log_base,
            mevis_args=mevis_args,
            merged_args=merged_args,
        )
else:
    with mlflow.start_run() as parent:
        train_procedure(
            parent,
            epoch,
            log_base=log_base,
            mevis_args=mevis_args,
            merged_args=merged_args,
        )

Number of trainable parameters in the model: 132352


Validation round: 100%|██████████| 10/10 [00:25<00:00,  2.57s/batch, loss (batch)=0.446]
2024/10/23 09:48:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Epoch 1:  49%|████▉     | 42/85 [03:21<03:26,  4.80s/batch, loss (batch)=0.525]
2024/10/23 09:51:24 INFO mlflow.tracking._tracking_service.client: 🏃 View run flawless-shark-609 at: http://0.0.0.0:5000/#/experiments/662304808085456660/runs/b1efbd001bfa4687a23386c0d2dbc57a.
2024/10/23 09:51:24 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://0.0.0.0:5000/#/experiments/662304808085456660.


KeyboardInterrupt: 

In [ ]:
with mlflow.start_run() as parent:
    
    base_save = CHECKPOINT_DIRECTORY / train_folder
    base_save.mkdir(parents=True, exist_ok=True)
    log_save = LOGDIR / train_folder
    log_save.mkdir(parents=True, exist_ok=True)

    # Log the hyperparameters
    mlflow.log_params(run_tags)

    # data
    dataset_validation = MRI_dataset_batched(
        args,
        data_file=VALID_DATA_FILE,
        batch_size=mevis_args.batch_size,
        phase="test",
        operation_mode="queue",
        mask_out_size=args.out_size,
        attention_size=64,
        crop=False,
        crop_size=1024,
        cls=1,
        if_prompt=True,
        prompt_type="point",
        if_attention_map=True,
        device=DEVICE,
    )
    dataset_train = MRI_dataset_batched(
        args,
        data_file=TRAIN_DATA_FILE,
        batch_size=mevis_args.batch_size,
        phase="train",
        operation_mode="queue",
        mask_out_size=args.out_size,
        attention_size=64,
        crop=False,
        crop_size=1024,
        cls=1,
        if_prompt=True,
        prompt_type="point",
        if_attention_map=True,
        device=DEVICE,
    )
    
    # model
    sam_mevis = build_sam_mevis(
        args,
        checkpoint=CHECKPOINT_DIRECTORY / "mobile_sam.pt",
        mevis_checkpoint=CHECKPOINT_DIRECTORY / BONE_CHECKPOINT,
        num_classes=2,
        device=DEVICE,
    )
    for param in sam_mevis.parameters():
        param.requires_grad = False
    for n, m in sam_mevis.mask_decoder.named_parameters():
        if "Adapter" in n:
            m.requires_grad = True

    total_trainable_params = sum(
        p.numel() for p in sam_mevis.parameters() if p.requires_grad
    )
    mlflow.log_param("total_trainable_params", total_trainable_params)
    print("Number of trainable parameters in the model:", total_trainable_params)

    lossfunc = DiceCELoss(sigmoid=True, squared_pred=True, reduction="mean")
    optimizer = torch.optim.AdamW(sam_mevis.parameters(), lr=mevis_args.warm)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")